In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [28]:
x = [[[2, -5, 3, 1, 5, 0],
     [8, 1, 0, 3, 3, 3],
     [-1, -1, -2, -1, -6, 0],
     [-3, 0, 6, -6, -1, 8],
     [0, -5, 1, 2, 1, -4],
     [-1, -3, 2, -3, -2, -4]]]

w1 = [[1, -1, 0],
      [-1, 2, -3],
      [0, -3, 1]]

w1_flipped = [[1, -3, 0],
              [-3, 2, -1],
              [0, -1, 1]]

w2_1 = [[-2, 1],
        [0, 2]]

w2_1_flipped = [[2, 0],
                [1, -2]]

w2_2 = [[-2, 0],
        [-1, 0]]

w2_2_flipped = [[0, -1],
                [0, -2]]

w2_3 = [[0, -1],
        [2, -1]]

w2_3_flipped = [[-1, 2],
                [-1, 0]]

w3 = [[-1, 1, -5],
      [2, -1, -1]]

In [29]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=(3, 3), stride=1)
        self.conv2 = nn.Conv2d(1, 3, kernel_size=(2, 2), stride=1)
        self.full = nn.Linear(3, 2)

        # Initialize weights
        # 卷积核需要翻转
        w1_flip = w1[::-1][::-1]
        self.conv1.weight.data = torch.tensor(w1_flipped).float().view(1, 1, 3, 3)
        self.conv2.weight.data = torch.tensor([w2_1_flipped, w2_2_flipped, w2_3_flipped]).float().view(3, 1, 2, 2)
        self.full.weight.data = torch.tensor(w3).float().view(2, 3)
        # Initialize biases
        self.conv1.bias.data = torch.tensor([0]).float()
        self.conv2.bias.data = torch.tensor([0, 0, 0]).float()
        self.full.bias.data = torch.tensor([0, 0]).float()

    def forward_print(self, x):
        x = self.conv1(x)
        print("conv1")
        print(x)

        x = F.relu(x)
        print("relu")
        print(x)

        x = F.max_pool2d(x, kernel_size=(2, 2), stride=2, padding=0)
        print("maxpool")
        print(x)

        x = self.conv2(x)
        print("conv2")
        print(x)

        x = F.relu(x)
        print("relu")
        print(x)

        x = self.full(x.view(x.size(0), -1))
        print("full")
        print(x)
        
        # softmax
        x = F.softmax(x, dim=1)
        print("softmax")
        print(x)
        return x

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=(2, 2), stride=2, padding=0)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.full(x.view(x.size(0), -1))
        # softmax
        x = F.softmax(x, dim=1)
        return x

In [30]:
print("Input:")
print(x)

x = torch.tensor(x).float().view(1, 1, 6, 6)
print("Input shape:", x.shape)
print("Input tensor:")
print(x)

print("Weights conv1:")
print(w1)
print("Weights conv2:")
print(w2_1, w2_2, w2_3)
print("Weights full:")
print(w3)

print("Bias conv1:")
print([0])
print("Bias conv2:")
print([0, 0, 0])

print("Bias full:")
print([0])

print("Output:")
model = cnn()
print(model(x))
print("Output shape:", model(x).shape)
print("Output tensor:")
print(model(x).detach().numpy())
print("Expected output:")

Input:
[[[2, -5, 3, 1, 5, 0], [8, 1, 0, 3, 3, 3], [-1, -1, -2, -1, -6, 0], [-3, 0, 6, -6, -1, 8], [0, -5, 1, 2, 1, -4], [-1, -3, 2, -3, -2, -4]]]
Input shape: torch.Size([1, 1, 6, 6])
Input tensor:
tensor([[[[ 2., -5.,  3.,  1.,  5.,  0.],
          [ 8.,  1.,  0.,  3.,  3.,  3.],
          [-1., -1., -2., -1., -6.,  0.],
          [-3.,  0.,  6., -6., -1.,  8.],
          [ 0., -5.,  1.,  2.,  1., -4.],
          [-1., -3.,  2., -3., -2., -4.]]]])
Weights conv1:
[[1, -1, 0], [-1, 2, -3], [0, -3, 1]]
Weights conv2:
[[-2, 1], [0, 2]] [[-2, 0], [-1, 0]] [[0, -1], [2, -1]]
Weights full:
[[-1, 1, -5], [2, -1, -1]]
Bias conv1:
[0]
Bias conv2:
[0, 0, 0]
Bias full:
[0]
Output:
tensor([[0.0025, 0.9975]], grad_fn=<SoftmaxBackward0>)
Output shape: torch.Size([1, 2])
Output tensor:
[[0.00247262 0.9975274 ]]
Expected output:


In [31]:
# 输出每一层的结果
print("Output:")
model.forward_print(x)

Output:
conv1
tensor([[[[ -6., -19.,  -2., -14.],
          [ 14., -11.,   6.,  -6.],
          [ 11.,  24., -29.,  20.],
          [ -9.,  -8.,  25.,  -5.]]]], grad_fn=<ConvolutionBackward0>)
relu
tensor([[[[ 0.,  0.,  0.,  0.],
          [14.,  0.,  6.,  0.],
          [11., 24.,  0., 20.],
          [ 0.,  0., 25.,  0.]]]], grad_fn=<ReluBackward0>)
maxpool
tensor([[[[14.,  6.],
          [24., 25.]]]], grad_fn=<MaxPool2DWithIndicesBackward0>)
conv2
tensor([[[[  2.]],

         [[-56.]],

         [[-26.]]]], grad_fn=<ConvolutionBackward0>)
relu
tensor([[[[2.]],

         [[0.]],

         [[0.]]]], grad_fn=<ReluBackward0>)
full
tensor([[-2.,  4.]], grad_fn=<AddmmBackward0>)
softmax
tensor([[0.0025, 0.9975]], grad_fn=<SoftmaxBackward0>)


tensor([[0.0025, 0.9975]], grad_fn=<SoftmaxBackward0>)